In [4]:
import pandas as pd
import numpy as np
import sklearn
from apos import appos
import string as st
import re
from nltk.corpus import stopwords 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from sklearn import naive_bayes
from sklearn.ensemble import RandomForestClassifier

In [5]:
a_file = open("english", "r")
lll= []
for line in a_file:
        stripped_line = line.strip()
        lll.append(stripped_line)


In [6]:
df=pd.read_csv("train.csv")

In [7]:
df=df.drop(['tweet_id'],axis=1)
df.head()

,sentiment,tweet_text
0,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,negative,Iranian general says Israel\u2019s Iron Dome c...
4,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...


In [8]:
def remove_slash(s):
    return s.replace('\\u2019',' ').replace('\\u002',' ')
def remove_punctuation(s):
    list_punctuation = [st.punctuation]
    for i in list_punctuation:
        s = s.replace(i,'')        
    return s
def replace_appos(s):
    for word in appos:
        if word in s:
            s.replace(word,appos[word])
    return s
def remove_tags(s):
    while(s.find('@')!=-1):
        if(s[s.find('@')+1:].find(' ')==-1):
            s=s[:s.find('@')]
        else:
            
            s=s[:s.find('@')]+s[s[s.find('@'):].find(' ')+s.find('@'):]
    return s
def mul_spaces(s):
    return re.sub(' +', ' ',s)

def stopwords_removal(data):
    stop_words = lll
    l=data.split(' ')
    g=l
    for i in l:
        if i in stop_words:
            g.remove(i)
    return ' '.join(l)
def remove_special_characters(text):
    pattern = r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

def lexicon_normalization(text):
    from nltk.stem.wordnet import WordNetLemmatizer
    lem = WordNetLemmatizer()
    return(lem.lemmatize(text,'n'))

def remove_links(s):
    while(s.find('http')!=-1):
        if(s[s.find('http')+1:].find(' ')==-1):
            s=s[:s.find('http')]
        else:
            
            s=s[:s.find('http')]+s[s[s.find('http'):].find(' ')+s.find('http'):]
    return s


In [9]:
df['tweet_text']=[i.lower() for i in df['tweet_text']]
df['tweet_text']=[remove_tags(i) for i in df['tweet_text']]
df['tweet_text']=[mul_spaces(i) for i in df['tweet_text']]
df['tweet_text']=[remove_slash(i) for i in df['tweet_text']]
df['tweet_text']=[replace_appos(i) for i in df['tweet_text']]
df['tweet_text']=[remove_punctuation(i) for i in df['tweet_text']]
df['tweet_text']=[stopwords_removal(i) for i in df['tweet_text']]
df['tweet_text']=[remove_special_characters(i) for i in df['tweet_text']]
df['tweet_text']=[lexicon_normalization(i) for i in df['tweet_text']]
df['tweet_text']=[remove_links(i) for i in df['tweet_text']]

In [10]:
df.head(10)

,sentiment,tweet_text
0,positive,gas my house hit m going chapel hill sat
1,negative,theo walcott still shit c watch rafa johnny de...
2,negative,not a gsp fan c i just hate nick diaz t wait f...
3,negative,iranian general says israel iron dome t deal t...
4,neutral,tehran c mon amour obama tried establish ties ...
5,neutral,sat this whole movie for harry ron christmas o...
6,positive,j davlar th main rivals team poland hopefully ...
7,negative,talking act sat c deciding i want go college ...
8,neutral,is \happy valentines day\ trending s the th fe...
9,negative,may superbowl dallas c dallas t winning a supe...


In [38]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', RandomForestClassifier())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'clf__alpha': [1, 1e-1, 1e-2]
}

In [39]:
x_train=df['tweet_text']
y_train=df['sentiment']

In [ ]:
from sklearn.metrics import classification_report
clf = GridSearchCV(text_clf, tuned_parameters, cv=10, scoring=None)
clf.fit(x_train, y_train)

In [23]:
test=pd.read_csv("test.csv")

In [24]:
x_test=test['tweet_text']

In [30]:
result_2=clf.predict(x_test)

In [41]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

tagged_data = [TaggedDocument(words=_d.split(' '), tags=[str(i)]) for i, _d in enumerate(df['tweet_text'])]

In [42]:
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


<ipython-input-42-f2c2304d9f55>:17: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  epochs=model.iter)


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

In [43]:
model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("shivank dadhwal is happy".lower())
if(model.infer_vector(test_data).all()>0):
    v1=model.infer_vector(test_data)
else:
    v1=-model.infer_vector(test_data)
v1

array([ 0.0093271 ,  0.04142823,  0.10919188,  0.19932699,  0.15781823,
        0.14754312,  0.01670253,  0.09030306, -0.03418628,  0.07104065,
        0.0113982 , -0.04633958,  0.11416581,  0.01215722,  0.03095124,
       -0.00217682, -0.20353381, -0.04949334, -0.02850072, -0.06990891],
      dtype=float32)

In [44]:
vecs=[]
for i in df['tweet_text']:
    vecs.append(model.infer_vector(word_tokenize(i)))


In [45]:
y=df['sentiment']
x=vecs

In [53]:
#we will train a RandomForest classifier
#rf=RandomForestClassifier()
#rf.fit(x,y)
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
clf.fit(x,y)
print(clf.score(x,y))

1.0


In [14]:
test=pd.read_csv("test.csv")

In [15]:
test.head()


,Unnamed: 0,tweet_id,tweet_text,vecs
0,0,264238274963451904,in atlantic city ventnor margate ocean city a...,[ 0.09703454 0.21580456 -0.17359503 -0.112266...
1,1,218775148495515649,musical awareness great big beautiful tomorrow...,[ 1.9649625e-01 2.6764107e-01 -3.3036727e-01 ...
2,2,258965201766998017,radio fm fri oct labour analyst shawn hattin...,[ 0.02543515 -0.04870443 0.10512904 -0.096038...
3,3,262926411352903682,kapan sih lo ngebuktiinjan ngomong doang susah...,[ 0.04412067 -0.05341641 0.01192275 0.015280...
4,4,171874368908050432,excuse connectivity this live stream baba amr ...,[ 0.17829536 0.11613123 -0.103672 -0.036830...


In [16]:
x_test=test['vecs']

In [17]:
print(x_test)

0       [ 0.09703454  0.21580456 -0.17359503 -0.112266...
1       [ 1.9649625e-01  2.6764107e-01 -3.3036727e-01 ...
2       [ 0.02543515 -0.04870443  0.10512904 -0.096038...
3       [ 0.04412067 -0.05341641  0.01192275  0.015280...
4       [ 0.17829536  0.11613123 -0.103672   -0.036830...
                              ...                        
5393    [ 0.13831736 -0.08691731 -0.18543796 -0.279293...
5394    [ 0.19730683  0.1614413  -0.16520138 -0.032636...
5395    [ 0.05194712  0.18074235 -0.16142698 -0.004499...
5396    [ 0.06033816  0.06531014 -0.02272399  0.037104...
5397    [ 0.05514403  0.14945884 -0.03649994 -0.114281...
Name: vecs, Length: 5398, dtype: object


In [54]:
pred_test=[]
for i in range(len(x_test)):
    prediction=clf.predict(x[i].reshape(1,-1))
    pred_test.append(prediction)
    

def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [55]:
result=np.array([])
for i in range(len(pred_test)):
    result=np.append(result,listToString(pred_test[i]))
print(result)

['positive' 'negative' 'negative' ... 'neutral' 'neutral' 'negative']


In [56]:
result1={
    'tweet_id': test['tweet_id'],
    'sentiment':result
}


data=pd.DataFrame(result1)

In [57]:
data.head(10)

,tweet_id,sentiment
0,264238274963451904,positive
1,218775148495515649,negative
2,258965201766998017,negative
3,262926411352903682,negative
4,171874368908050432,neutral
5,256010056942903296,neutral
6,253809989599232000,positive
7,261776619146985472,negative
8,264143999374356481,neutral
9,223052929131757571,negative


In [58]:
data.to_csv('final.csv',index=False) 